## Building a recurrent neural network with PyTorch (GPU)

## Model C: 2 Hidden Layer (tanH)

Chose tanH because it had the highest accuracy when we ran the various models on CPU.

In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.autograd import Variable

When running on GPU, remember two things must live on the GPU
* Variables
* Model itself
### Steps
* STEP 1: Load dataset
* STEP 2: Make dataset iterable
* **STEP 3: Create model class**
* **STEP 4: Instantiate model class**
* STEP 5: Instantiate loss class
* STEP 6: Instantiate optimizer class
* **STEP 7: Train Model!**

In [2]:
'''
STEP 1: Load Dataset
'''
train_dataset = dsets.MNIST(root="./data",
                            train=True,
                            transform=transforms.ToTensor(), 
                            download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

'''
STEP 2: Make Dataset Iterable
'''

batch_size = 100
n_iters = 3000
num_epochs = int(n_iters / (len(train_dataset)/batch_size))

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

'''
STEP 3: Create Model Class
'''

class RNNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(RNNModel, self).__init__()
        # Hidden Dimensions
        self.hidden_dim = hidden_dim
        
        # Number of hidden layers
        self.layer_dim = layer_dim
        
        # Building your RNN
        # batch_first=True causes input/output Tensors to be of shape
        # (batch_dim, seq_dim, input_dim)
        self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='tanh')
        
        # Readout Layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        # Initialize hidden state with zeros
        # (layer_dim, batch_size, hidden_dim)
        ########################
        # Move Variable to GPU #
        ########################
        if torch.cuda.is_available():
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).cuda())
        else:
            h0 = Variable(torch.zeros(self.layer_dim, x.size(0), self.hidden_dim))   
        
        # One time step
        out, hn = self.rnn(x, h0)
        
        # Index hidden state of last time step
        # out.size() --> 100, 28, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states!
        out = self.fc(out[:, -1, :])
        # out.size() --> 100, 10
        return out

'''
STEP 4: Instantiate Model Class
'''

input_dim = 28
hidden_dim = 100
layer_dim = 2   # This is the only change to add another hidden layer to the model! Simple! 
output_dim = 10

model = RNNModel(input_dim, hidden_dim, layer_dim, output_dim)
#####################
# Use GPU for Model #
#####################
if torch.cuda.is_available():
    model.cuda()

# Printing model and parameters to illustrate the difference with our 2 hidden layer neural network

print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())
    
'''
STEP 5: Instantiate Loss Class
'''

criterion = nn.CrossEntropyLoss()

'''
STEP 6: Instantiate Optimizer Class
'''

learning_rate = 0.1
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

'''
STEP 7: Train the Model
'''

# Number of steps to unroll
seq_dim = 28

iter = 0
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        #####################
        # Use GPU for Model #
        #####################
        if torch.cuda.is_available():
            images = Variable(images.view(-1, seq_dim, input_dim).cuda()) 
            labels = Variable(labels.cuda())
        else:
            images = Variable(images.view(-1, seq_dim, input_dim)) 
            labels = Variable(labels)
        
        #Clear gradients w.r.t. parameters
        optimizer.zero_grad()
        
        #Forward pass to get outputs / logits
        # outputs.size() --> (100, 10)
        outputs = model(images)
        
        #Calculate Loss: softmax --> Cross Entropy Loss
        loss = criterion(outputs, labels)
        
        #Get gradients w.r.t. parameters
        loss.backward()
        
        #Update parameters
        optimizer.step()
        
        iter += 1
        if iter % 500 == 0:
            # Calculate Accuracy
            correct = 0 
            total = 0
            #Iterate through the test dataset
            for images, labels in test_loader:
                # Load images to Torch Variable
                #####################
                # Use GPU for Model #
                #####################
                if torch.cuda.is_available():
                    images = Variable(images.view(-1, seq_dim, input_dim).cuda())
                else:
                    images = Variable(images.view(-1, seq_dim, input_dim))
                
                # Forward pass only to get outputs/logits
                outputs = model(images)
                
                # Get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels
                total += labels.size(0)
                
                # Total correct predictions
                #####################
                # Use GPU for Model # Now we have to bring predicted back to the CPU
                #####################
                if torch.cuda.is_available():
                    correct += (predicted.cpu() == labels.cpu()).sum()
                else:
                    correct += (predicted == labels).sum()
                    
            accuracy = 100 * correct / total
            
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data[0], accuracy))

RNNModel (
  (rnn): RNN(28, 100, num_layers=2, batch_first=True)
  (fc): Linear (100 -> 10)
)
10
torch.Size([100, 28])
torch.Size([100, 100])
torch.Size([100])
torch.Size([100])
torch.Size([100, 100])
torch.Size([100, 100])
torch.Size([100])
torch.Size([100])
torch.Size([10, 100])
torch.Size([10])
Iteration: 500. Loss: 1.1273566484451294. Accuracy: 63.92
Iteration: 1000. Loss: 0.37306421995162964. Accuracy: 90.01
Iteration: 1500. Loss: 0.14018236100673676. Accuracy: 94.16
Iteration: 2000. Loss: 0.1459290087223053. Accuracy: 95.12
Iteration: 2500. Loss: 0.18473494052886963. Accuracy: 95.23
Iteration: 3000. Loss: 0.3049411475658417. Accuracy: 93.42


## Summary of RNN

* Essentially, RNN's are like FNN (feedforward)
* Allow us to take this knowledge and apply it to the next type of NN:
    * Long Short Term Memory (LSTM) neural networks are based on RNN's